In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import copy

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("pinkmanlove/llama-7b-hf")
model = AutoModelForCausalLM.from_pretrained("pinkmanlove/llama-7b-hf", torch_dtype = torch.bfloat16)

In [ ]:
for x in model.base_model.layers[31].self_attn.named_parameters():
        print(x[1].shape)

In [ ]:
origin_model = copy.deepcopy(model)

In [ ]:
tokenizer.encode('你好')

In [ ]:
origin_model.forward(torch.tensor([[1, 29871, 30919, 31076]]))

In [ ]:
import ast
heads_to_prune = ast.literal_eval("{31:[0,1,2,3,4,5,6,7,8]}")

In [ ]:
model.prune_heads({31:[0,1,2,3,4,5,6,7,8],30:[0,1,2,3,4,5,6,7,8],29:[0,1,2,3,4,5,6,7,8],28:[0,1,2,3,4,5,6,7,8]})

In [ ]:
model.forward(torch.tensor([[1, 29871, 30919, 31076]]))

In [ ]:
for x in model.base_model.layers[31].self_attn.named_parameters():
    print(x[1].shape)

In [ ]:
model.save_pretrained("pruned_5heads")

In [ ]:
model_pruned = AutoModelForCausalLM.from_pretrained("/home/zhangyihan/projects/LMFlow/output_models/llama-7b-mha")

In [ ]:
print(model.base_model.num_parameters())
print(model_pruned.base_model.num_parameters())

In [1]:
import json

from transformers import HfArgumentParser
import torch
from lmflow.datasets.dataset import Dataset
from lmflow.pipeline.auto_pipeline import AutoPipeline
from lmflow.models.auto_model import AutoModel
from lmflow.args import ModelArguments, DatasetArguments, AutoArguments
import deepspeed


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/zhangyihan/anaconda3/envs/lmflow/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/zhangyihan/anaconda3/envs/lmflow/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/zhangyihan/anaconda3/envs/lmflow/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /home/zhangyihan/anaconda3/envs/lmflow did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/zhangyihan/anaconda3/envs/lmflow/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/zhangyihan/anaconda3/envs/lmflow/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home/zhangyihan/anaconda3/envs/lmflow/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcuda

In [2]:
pipeline_name = "finetuner"
PipelineArguments = AutoArguments.get_pipeline_args_class(pipeline_name)
model_args = ModelArguments(model_name_or_path='/home/zhangyihan/projects/LMFlow/output_models/llama-13b-mha40x10',torch_dtype='bfloat16')
data_args = DatasetArguments(dataset_path='/home/zhangyihan/projects/LMFlow/data/lmflow_data')

In [3]:
pipeline_args = PipelineArguments(deepspeed='../configs/ds_config_zero3.json',bf16=True,do_train=True,num_train_epochs=1,output_dir='output_models')

[2023-08-08 13:50:46,683] [INFO] [comm.py:634:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2023-08-08 13:50:46,806] [INFO] [comm.py:688:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=192.168.97.252, master_port=29500
[2023-08-08 13:50:46,807] [INFO] [comm.py:652:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


In [4]:
finetuner = AutoPipeline.get_pipeline(
        pipeline_name=pipeline_name,
        model_args=model_args,
        data_args=data_args,
        pipeline_args=pipeline_args,
    )

08/08/2023 06:55:13 - WARNING - lmflow.pipeline.finetuner - Process rank: 0, device: cuda:0, n_gpu: 1,distributed training: True, 16-bits training: False


In [2]:
model = AutoModel.get_model(model_args)

NameError: name 'model_args' is not defined

In [6]:
list(model.get_backend_model().base_model.layers[0].named_parameters())[0]

('self_attn.q_proj.weight',
 Parameter containing:
 tensor([], device='cuda:0', dtype=torch.bfloat16, requires_grad=True))

In [ ]:
deepspeed.zero.register_external_parameter(model.get_backend_model().base_model.layers[0],model.get_backend_model().base_model.layers[0].get_parameter('self_attn.q_proj.weight'))

In [2]:
model_args = ModelArguments(model_name_or_path='pinkmanlove/llama-7b-hf',torch_dtype='bfloat16')
model = AutoModel.get_model(model_args)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
print(model.get_backend_model().base_model.num_parameters())

6607343616


In [11]:
model.get_backend_model().forward(torch.tensor([[1]]))

RuntimeError: 'weight' must be 2-D